In [1]:
#install nltk
!pip install nltk

In [2]:
!python -m nltk.downloader popular

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_d

In [3]:
artikel = "Ketua DPP Partai Persatuan Indonesia (Perindo) Bidang ESDM Andi Asmara. Andi Asmara merupakan seorang pengusaha batu bara dari Sumatera Selatan yang cukup dikenal di daerahnya. Andi saat ini menjabat sebagai Ketua Asosiasi Batubara Sumatera Selatan. Selain itu, Andi Asmara tercatat menjabat sebagai Komisaris Utama PT Anda Mas Global Energi. PT Andamas Global Energi didirikan Andi Asmara, seorang pelopor sekaligus pionir industri batubara swasta di daerah Kabupaten Lahat, Sumatera Selatan sejak tahun 2008. Di Pileg 2024, Andi Asmara telah memantapkan hati untuk mewakili daerahnya sebagai calon anggota DPR RI."

In [4]:
print(artikel)

Ketua DPP Partai Persatuan Indonesia (Perindo) Bidang ESDM Andi Asmara. Andi Asmara merupakan seorang pengusaha batu bara dari Sumatera Selatan yang cukup dikenal di daerahnya. Andi saat ini menjabat sebagai Ketua Asosiasi Batubara Sumatera Selatan. Selain itu, Andi Asmara tercatat menjabat sebagai Komisaris Utama PT Anda Mas Global Energi. PT Andamas Global Energi didirikan Andi Asmara, seorang pelopor sekaligus pionir industri batubara swasta di daerah Kabupaten Lahat, Sumatera Selatan sejak tahun 2008. Di Pileg 2024, Andi Asmara telah memantapkan hati untuk mewakili daerahnya sebagai calon anggota DPR RI.


## Summarization

In [5]:
class ItemsCount(object):
    def __init__(self, value):
        self._value = value

    def __call__(self, sequence):
        if isinstance(self._value, (bytes, str,)):
            if self._value.endswith("%"):
                total_count = len(sequence)
                percentage = int(self._value[:-1])
                # setidaknya 1 kalimat harus dipilih
                count = max(1, total_count*percentage // 100)
                return sequence[:count]
            else:
                return sequence[:int(self._value)]
        elif isinstance(self._value, (int, float)):
            return sequence[:int(self._value)]
        else:
            ValueError("Unsuported value of items count '%s'." % self._value)

    def __repr__(self):
        return to_string("<ItemsCount: %r>" % self._value)

In [6]:
from operator import attrgetter
from collections import namedtuple

SentenceInfo = namedtuple("SentenceInfo", ("sentence", "order", "rating",))

class BaseSummarizer(object):

    def __call__(self, output, sentences_count):
        raise NotImplementedError("This method should be overriden in subclass")

    @staticmethod
    def normalize_word(word):
        return word.lower()

    @staticmethod
    def _get_best_sentences(sentences, count, rating, *args, **kwargs):
        rate = rating
        if isinstance(rating, dict):
            assert not args and not kwargs
            rate = lambda s: rating[s]

        infos = (SentenceInfo(s, o, rate(s, *args, **kwargs))
            for o, s in enumerate(sentences))

        # sort kalimat berdasarkan secara descending
        infos = sorted(infos, key=attrgetter("rating"), reverse=True)
        # get `count` first best rated sentences
        if not isinstance(count, ItemsCount):
            count = ItemsCount(count)
        infos = count(infos)
        # sort mengurutkan kalimat berdasarkan urutannya dalam dokumen
        infos = sorted(infos, key=attrgetter("order"))

        return tuple(i.sentence for i in infos)

In [7]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
import math
import numpy
import nltk

from warnings import warn
from nltk.tokenize import sent_tokenize, word_tokenize
from numpy.linalg import svd as singular_value_decomposition
from nltk.corpus import stopwords

class LsaSummarizer(BaseSummarizer):
    MIN_DIMENSIONS = 3
    REDUCTION_RATIO = 1/1

    _stop_words = list(stopwords.words('indonesian'))

    @property
    def stop_words(self):
        return self._stop_words

    @stop_words.setter
    def stop_words(self, words):
        self._stop_words = words

    def __call__(self, output, sentences_count):

        dictionary = self._create_dictionary(artikel)

        if not dictionary:
            return ()

        sentences = sent_tokenize(artikel)

        matrix = self._create_matrix(artikel, dictionary)
        matrix = self._compute_term_frequency(matrix)
        u, sigma, v = singular_value_decomposition(matrix, full_matrices=False) #svd

        ranks = iter(self._compute_ranks(sigma, v))
        return self._get_best_sentences(sentences, sentences_count,
            lambda s: next(ranks))

    def _create_dictionary(self, artikel):
        """mapping key = kata, nilai = indeks baris"""

        words = word_tokenize(artikel)
        words = tuple(words)

        words = map(self.normalize_word, words)

        #unique_words = ("Tari_Sajojo", "Ki_Anom_Suroto", "Anom_Suroto", "Ki_Nartosando", "Sujiwo_Tedjo", "Ki_Suwarto" , "Enthus_Susmono", "Slamet_Gundono", "Sujiwo_Tedjo", "Ki_Suwarto_Kondobuwono", "Hanung_Anindita_Wijayasakti", "Gineng_Pratidina_Permana_Sakti", "Asep_Sunandar", "Eyang_Abiyasa", "Ki_Enthus_Susmono","Soeharto", "Ki_Timbul_Hadiprayitno" , "Ki_Timbul", "Abdurrahman_Wahid", "Gito_Wasito", "Sinah", "Soenarto", "Partinoyo", "Ki_Narto" , "Dr._Sri_Teddy_Rusdy,S.H.,M.Hum.M,")
        unique_words = frozenset(w for w in words if w not in self._stop_words)
        return dict((w, i) for i, w in enumerate(unique_words))

    def _create_matrix(self, artikel, dictionary):
        """
        Membuat matriks bentuk di mana sel berisi jumlah kemunculan kata (baris) dalam kalimat (kolom).
        """
        sentences = sent_tokenize(artikel)
        words_count = len(dictionary)
        sentences_count = len(sentences)
        if words_count < sentences_count:
            message = (
                "Jumlah kata (%d) lebih sedikit daripada jumlah kalimat (%d). "
                "Algoritma LSA mungkin tidak bekerja dengan baik."
            )
            warn(message % (words_count, sentences_count))

        matrix = numpy.zeros((words_count, sentences_count))
        for col, sentence in enumerate(sentences):
            words = word_tokenize(sentence)
            for word in words:
                # hanya valid words yg terhitung (not stop-words, ...)
                if word in dictionary:
                    row = dictionary[word]
                    matrix[row, col] = 1

        return matrix

    def _compute_term_frequency(self, matrix, smooth=0.4):
        assert 0.0 <= smooth < 1.0

        max_word_frequencies = numpy.max(matrix, axis=0)
        rows, cols = matrix.shape
        for row in range(rows):
            for col in range(cols):
                max_word_frequency = max_word_frequencies[col]
                if max_word_frequency != 0:
                    frequency = matrix[row, col]/max_word_frequency
                    matrix[row, col] = smooth + (1.0 - smooth)*frequency

        return matrix

    def _compute_ranks(self, sigma, v_matrix):
        assert len(sigma) == v_matrix.shape[0]

        dimensions = max(LsaSummarizer.MIN_DIMENSIONS,
            int(len(sigma)*LsaSummarizer.REDUCTION_RATIO))
        powered_sigma = tuple(s**2 if i < dimensions else 0.0
            for i, s in enumerate(sigma))

        ranks = []

        for column_vector in v_matrix.T:
            rank = sum(s*v**2 for s, v in zip(powered_sigma, column_vector))
            ranks.append(math.sqrt(rank))

        return ranks

In [8]:
import nltk
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

from nltk.corpus import stopwords

source_file = "artikel"

text = artikel

summarizer = LsaSummarizer()

stopwords = set(stopwords.words('indonesian'))
summarizer.stop_words = stopwords
summary = summarizer(artikel, 3)

print("======asli =====")
print(text)
print("====== Akhir dari text asli =====")



print("\n========= Ringkasan =========")
hasil_sum = (" ".join(summary))
print(hasil_sum)
print("========= Akhir ringkasan =========")


======asli =====
Ketua DPP Partai Persatuan Indonesia (Perindo) Bidang ESDM Andi Asmara. Andi Asmara merupakan seorang pengusaha batu bara dari Sumatera Selatan yang cukup dikenal di daerahnya. Andi saat ini menjabat sebagai Ketua Asosiasi Batubara Sumatera Selatan. Selain itu, Andi Asmara tercatat menjabat sebagai Komisaris Utama PT Anda Mas Global Energi. PT Andamas Global Energi didirikan Andi Asmara, seorang pelopor sekaligus pionir industri batubara swasta di daerah Kabupaten Lahat, Sumatera Selatan sejak tahun 2008. Di Pileg 2024, Andi Asmara telah memantapkan hati untuk mewakili daerahnya sebagai calon anggota DPR RI.
====== Akhir dari text asli =====

========= Ringkasan =========
Andi Asmara merupakan seorang pengusaha batu bara dari Sumatera Selatan yang cukup dikenal di daerahnya. PT Andamas Global Energi didirikan Andi Asmara, seorang pelopor sekaligus pionir industri batubara swasta di daerah Kabupaten Lahat, Sumatera Selatan sejak tahun 2008. Di Pileg 2024, Andi Asmara te